In [1]:
import numpy as np
import pandas as pd

## Bug Demo

In [2]:
train = pd.read_csv('data/train_0512.csv')
valid = pd.read_csv('data/valid_0512.csv')
test = pd.read_csv('data/test_0512.csv')

In [3]:
conv_id_counts = train.value_counts('conv_id')
conv_id_counts[conv_id_counts == 1]

conv_id
hit:7600_conv:15200     1
hit:818_conv:1637       1
hit:3397_conv:6794      1
hit:11439_conv:22879    1
hit:10015_conv:20030    1
                       ..
hit:558_conv:1117       1
hit:7881_conv:15763     1
hit:2257_conv:4515      1
hit:11151_conv:22303    1
hit:826_conv:1653       1
Length: 64, dtype: int64

In [4]:
conv_id_counts = valid.value_counts('conv_id')
conv_id_counts[conv_id_counts == 1]

conv_id
hit:4299_conv:8599      1
hit:9594_conv:19188     1
hit:10142_conv:20284    1
hit:5771_conv:11543     1
hit:10931_conv:21862    1
dtype: int64

In [5]:
conv_id_counts = test.value_counts('conv_id')
conv_id_counts[conv_id_counts == 1]

conv_id
hit:9648_conv:19297    1
hit:2803_conv:5607     1
dtype: int64

In [6]:
train[train['conv_id'] == 'hit:2444_conv:4888']['utterance'].values

array(['i was happy when Lebron signed with the lakers,5|5|5_4|5|5,\nhit:2444_conv:4888,2,joyful,i was happy when Lebron signed with the lakers,390,Oh yeah! So was I. That was so awesome! ,5|5|5_4|5|5,\nhit:2444_conv:4888,3,joyful,i was happy when Lebron signed with the lakers,238,yea i hope they take down the warriors,5|5|5_4|5|5,\nhit:2444_conv:4888,4,joyful,i was happy when Lebron signed with the lakers,390,Woo hoo! That would be so awesome! ,5|5|5_4|5|5,\nhit:2444_conv:4889,1,anxious,I was on my way to the eye doctor yesterday and thought I left on time. I wound up being 15 minutes late despite frantically driving like a madman. ,390,I get so frantic when I am late for appointments. I had an eye doctor appointment and was 15 minutes late. ,4|5|5_5|5|5,\nhit:2444_conv:4889,2,anxious,I was on my way to the eye doctor yesterday and thought I left on time. I wound up being 15 minutes late despite frantically driving like a madman. ,238,what ended up happening,4|5|5_5|5|5,\nhit:2444_con

In [7]:
valid[valid['conv_id'] == 'hit:4299_conv:8599']['utterance'].values

array(['I\'m devastated by my mother\'s death,5|5|5_5|5|5,\nhit:4299_conv:8599,2,devastated,I\'m devastated by my mother\'s death,294,I am so sorry to hear that she has passed.  How long has she been gone?,5|5|5_5|5|5,,I love it when it\'s nice out. Are you going to do something outside?|Save it for Spider-Man that comes out next month!  I can\'t wait for that|I never took chemistry. Do you feel prepared for the test?|Yea open book make you think you dont need to study but that isnt true at all|Oh_comma_ that must have been so precious! We had deer around where I used to live - they\'re gorgeous creatures.|It\'s ok to have a cheat day everyone once in a while_comma_ but you can\'t give in. Be strong!|Oh no! Well hopefully she wasn\'t too upset_comma_ then you\'ll know how her character is :)|oh yea whys that?|Oh that sucks! What did your neighbor do to get you angry?|Someone\'s gotta put an end to that poor service!|Sorry about that_comma_ you could have taken a taxi|That is good_comma

In [8]:
test[test['conv_id'] == 'hit:9648_conv:19297']['utterance'].values

array(['I am by myself most of the time in a isolated area. I only have animals as my friends.,5|5|4_5|5|5,\nhit:9648_conv:19297,2,lonely,I am by myself most of the time in a isolated area. I only have animals as my friends. ,568,oh does that sadden you?,5|5|4_5|5|5,,You can track her down to say sorry_comma_ or would that be creepy?|One day we\'ll find a way to repay them all the misery they\'ve caused us.|I see_comma_ did they ever find out it was you ?|Well_comma_ if the drive was the most challenging it sounds like a good trip! Too often it\'s family dynamics that are *more* challenging -- and not so easily taken care of. ;-)|That is awesome. I am sure you will return the favor one day.|holy Moses! I hate them things. Flying rats is what they are.|That\'s good_comma_ at least he is getting better_comma_ he\'ll survive!|That is really creepy_comma_ I would feel anxious too. I think you should report the to the police if you notice any strange or threatening messages_comma_Feel free 

## Bug Fix

In [9]:
# the raw data is from https://github.com/facebookresearch/EmpatheticDialogues, 
# which is the code of the paper mentioned in TA's ppt
!wget "https://dl.fbaipublicfiles.com/parlai/empatheticdialogues/empatheticdialogues.tar.gz";
!tar xzvf empatheticdialogues.tar.gz;
!rm -f empatheticdialogues.tar.gz

--2022-05-13 19:29:50--  https://dl.fbaipublicfiles.com/parlai/empatheticdialogues/empatheticdialogues.tar.gz
已載入 CA 憑證「/etc/ssl/certs/ca-certificates.crt」
正在查找主機 dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
正在連接 dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... 連上了。
已送出 HTTP 要求，正在等候回應... 200 OK
長度: 28022709 (27M) [application/gzip]
儲存到：「empatheticdialogues.tar.gz」

empatheticdialogues 100%[===================>]  26.72M  8.46MB/s  於 3.6s       

2022-05-13 19:29:55 (7.32 MB/s) - 已儲存 「empatheticdialogues.tar.gz」 [28022709/28022709]

empatheticdialogues/
empatheticdialogues/test.csv
empatheticdialogues/train.csv
empatheticdialogues/valid.csv


In [10]:
sent_id = {
    'sad':      0,  'trusting':     1,  'terrified': 2,  'caring':      3,  'disappointed': 4, 
    'faithful': 5,  'joyful':       6,  'jealous':   7,  'disgusted':   8,  'surprised':    9, 
    'ashamed':  10, 'afraid':       11, 'impressed': 12, 'sentimental': 13, 'devastated':   14, 
    'excited':  15, 'anticipating': 16, 'annoyed':   17, 'anxious':     18, 'furious':      19, 
    'content':  20, 'lonely':       21, 'angry':     22, 'confident':   23, 'apprehensive': 24, 
    'guilty':   25, 'embarrassed':  26, 'grateful':  27, 'hopeful':     28, 'proud':        29, 
    'prepared': 30, 'nostalgic':    31
}

def fixed_data():
    fixed_data = []
    for fn in ['train', 'valid', 'test']:
        with open(f'empatheticdialogues/{fn}.csv', mode='r', encoding='utf8') as f:
            lines = f.readlines()
            cols = lines[0].strip().split(',')
            
            cells = []
            for line in lines[1:]:
                row = line.strip().split(',')
                if len(row) > len(cols):
                    # exclude the redundant col
                    row = row[:(len(cols) - len(row))]
                    
                cells.append(row)
            
            df = pd.DataFrame(data=cells, columns=cols)
            
            df.drop(['speaker_idx', 'selfeval', 'tags'], axis=1, inplace=True)
            if fn != 'test':
                df['label'] = list(map(lambda x: sent_id[x], df['context']))
            df.drop(['context'], axis=1, inplace=True)
            
            print(f'{fn} data finished.')
            fixed_data.append(df)
            
    return fixed_data

In [11]:
fixed_train, fixed_valid, fixed_test = fixed_data()

train data finished.
valid data finished.
test data finished.


In [12]:
# validate the fixed data
display(fixed_train)
display(fixed_valid)
display(fixed_test)

,conv_id,utterance_idx,prompt,utterance,label
0,hit:0_conv:1,1,I remember going to the fireworks with my best...,I remember going to see the fireworks with my ...,13
1,hit:0_conv:1,2,I remember going to the fireworks with my best...,Was this a friend you were in love with_comma_...,13
2,hit:0_conv:1,3,I remember going to the fireworks with my best...,This was a best friend. I miss her.,13
3,hit:0_conv:1,4,I remember going to the fireworks with my best...,Where has she gone?,13
4,hit:0_conv:1,5,I remember going to the fireworks with my best...,We no longer talk.,13
...,...,...,...,...,...
84164,hit:12424_conv:24848,5,I found some pictures of my grandma in the att...,Yeah reminds me of the good old days. I miss ...,13
84165,hit:12424_conv:24849,1,I woke up this morning to my wife telling me s...,I woke up this morning to my wife telling me s...,9
84166,hit:12424_conv:24849,2,I woke up this morning to my wife telling me s...,Oh hey that's awesome! That is awesome right?,9
84167,hit:12424_conv:24849,3,I woke up this morning to my wife telling me s...,It is soooo awesome. We have been wanting a b...,9


,conv_id,utterance_idx,prompt,utterance,label
0,hit:3_conv:6,1,Today_comma_as i was leaving for work in the m...,Today_comma_as i was leaving for work in the m...,2
1,hit:3_conv:6,2,Today_comma_as i was leaving for work in the m...,Are you fine now?,2
2,hit:3_conv:6,3,Today_comma_as i was leaving for work in the m...,Yeah_comma_i'm doing alright now_comma_ but wi...,2
3,hit:3_conv:6,4,Today_comma_as i was leaving for work in the m...,Cool :) Is your car damaged a lot?,2
4,hit:3_conv:6,5,Today_comma_as i was leaving for work in the m...,The car was badly damaged_comma_i veered outsi...,2
...,...,...,...,...,...
12073,hit:12361_conv:24722,4,One time I studied all night for my final exam!,tha is really cool what was your grade,30
12074,hit:12392_conv:24785,1,One of my coworkers has been arguing with his ...,One of my coworkers has been arguing with his ...,19
12075,hit:12392_conv:24785,2,One of my coworkers has been arguing with his ...,What are they arguing about?,19
12076,hit:12392_conv:24785,3,One of my coworkers has been arguing with his ...,Everything and anything. It's annoying_comma_ ...,19


,conv_id,utterance_idx,prompt,utterance
0,hit:0_conv:0,1,I felt guilty when I was driving home one nigh...,Yeah about 10 years ago I had a horrifying exp...
1,hit:0_conv:0,2,I felt guilty when I was driving home one nigh...,Did you suffer any injuries?
2,hit:0_conv:0,3,I felt guilty when I was driving home one nigh...,No I wasn't hit. It turned out they were drunk...
3,hit:0_conv:0,4,I felt guilty when I was driving home one nigh...,Why did you feel guilty? People really shouldn...
4,hit:0_conv:0,5,I felt guilty when I was driving home one nigh...,I don't know I was new to driving and hadn't e...
...,...,...,...,...
10968,hit:12416_conv:24832,4,I saw a huge cockroach outside my house today....,I live in Texas to so i know those feels
10969,hit:12423_conv:24847,1,I have a big test on Monday. I am so nervous_c...,I have a big test on Monday_comma_ I am so ner...
10970,hit:12423_conv:24847,2,I have a big test on Monday. I am so nervous_c...,What is the test on?
10971,hit:12423_conv:24847,3,I have a big test on Monday. I am so nervous_c...,It's for my Chemistry class. I haven't slept m...


In [13]:
display(train)
display(valid)
display(test)

,id,conv_id,utterance_idx,prompt,utterance,label
0,0,hit:0_conv:1,1,I remember going to the fireworks with my best...,I remember going to see the fireworks with my ...,13
1,1,hit:0_conv:1,2,I remember going to the fireworks with my best...,Was this a friend you were in love with_comma_...,13
2,2,hit:0_conv:1,3,I remember going to the fireworks with my best...,This was a best friend. I miss her.,13
3,3,hit:0_conv:1,4,I remember going to the fireworks with my best...,Where has she gone?,13
4,4,hit:0_conv:1,5,I remember going to the fireworks with my best...,We no longer talk.,13
...,...,...,...,...,...,...
76668,76668,hit:12424_conv:24848,5,I found some pictures of my grandma in the att...,Yeah reminds me of the good old days. I miss ...,13
76669,76669,hit:12424_conv:24849,1,I woke up this morning to my wife telling me s...,I woke up this morning to my wife telling me s...,9
76670,76670,hit:12424_conv:24849,2,I woke up this morning to my wife telling me s...,Oh hey that's awesome! That is awesome right?,9
76671,76671,hit:12424_conv:24849,3,I woke up this morning to my wife telling me s...,It is soooo awesome. We have been wanting a b...,9


,id,conv_id,utterance_idx,prompt,utterance,label
0,0,hit:3_conv:6,1,Today_comma_as i was leaving for work in the m...,Today_comma_as i was leaving for work in the m...,2
1,1,hit:3_conv:6,2,Today_comma_as i was leaving for work in the m...,Are you fine now?,2
2,2,hit:3_conv:6,3,Today_comma_as i was leaving for work in the m...,Yeah_comma_i'm doing alright now_comma_ but wi...,2
3,3,hit:3_conv:6,4,Today_comma_as i was leaving for work in the m...,Cool :) Is your car damaged a lot?,2
4,4,hit:3_conv:6,5,Today_comma_as i was leaving for work in the m...,The car was badly damaged_comma_i veered outsi...,2
...,...,...,...,...,...,...
12025,12025,hit:12361_conv:24722,4,One time I studied all night for my final exam!,tha is really cool what was your grade,30
12026,12026,hit:12392_conv:24785,1,One of my coworkers has been arguing with his ...,One of my coworkers has been arguing with his ...,19
12027,12027,hit:12392_conv:24785,2,One of my coworkers has been arguing with his ...,What are they arguing about?,19
12028,12028,hit:12392_conv:24785,3,One of my coworkers has been arguing with his ...,Everything and anything. It's annoying_comma_ ...,19


,id,conv_id,utterance_idx,prompt,utterance
0,0,hit:0_conv:0,1,I felt guilty when I was driving home one nigh...,Yeah about 10 years ago I had a horrifying exp...
1,1,hit:0_conv:0,2,I felt guilty when I was driving home one nigh...,Did you suffer any injuries?
2,2,hit:0_conv:0,3,I felt guilty when I was driving home one nigh...,No I wasn't hit. It turned out they were drunk...
3,3,hit:0_conv:0,4,I felt guilty when I was driving home one nigh...,Why did you feel guilty? People really shouldn...
4,4,hit:0_conv:0,5,I felt guilty when I was driving home one nigh...,I don't know I was new to driving and hadn't e...
...,...,...,...,...,...
10938,10938,hit:12416_conv:24832,4,I saw a huge cockroach outside my house today....,I live in Texas to so i know those feels
10939,10939,hit:12423_conv:24847,1,I have a big test on Monday. I am so nervous_c...,I have a big test on Monday_comma_ I am so ner...
10940,10940,hit:12423_conv:24847,2,I have a big test on Monday. I am so nervous_c...,What is the test on?
10941,10941,hit:12423_conv:24847,3,I have a big test on Monday. I am so nervous_c...,It's for my Chemistry class. I haven't slept m...


In [14]:
conv_id_counts = fixed_train.value_counts('conv_id')
conv_id_counts[conv_id_counts == 1]

conv_id
hit:6_conv:13       1
hit:296_conv:593    1
dtype: int64

In [15]:
display(fixed_train[fixed_train['conv_id'] == 'hit:6_conv:13']['utterance'].values)
display(fixed_train[fixed_train['conv_id'] == 'hit:296_conv:593']['utterance'].values)

array(['The other night I was alone and heard a nose coming from the kitchen... it was creepy. '],
      dtype=object)

array(['I feel at school in front of the whole class.'], dtype=object)

In [16]:
conv_id_counts = fixed_valid.value_counts('conv_id')
conv_id_counts[conv_id_counts == 1]

conv_id
hit:5771_conv:11543    1
dtype: int64

In [17]:
display(fixed_valid[fixed_valid['conv_id'] == 'hit:5771_conv:11543']['utterance'].values)

array(['Oh. But you are still here...'], dtype=object)

In [18]:
conv_id_counts = fixed_test.value_counts('conv_id')
conv_id_counts[conv_id_counts == 1]

Series([], dtype: int64)

In [20]:
fixed_train.to_csv('data/fixed_train.csv', index=False, encoding='utf8')
fixed_valid.to_csv('data/fixed_valid.csv', index=False, encoding='utf8')
fixed_test.to_csv('data/fixed_test.csv', index=False, encoding='utf8')